# Topic 5. Recursive functions and data types

The goals of this topic are to understand:

* How recursive types (lists, trees, etc.) are defined algebraically
* How functions over recursive types are defined recursivelly
* The two major types of recursive functions: general and tail-recursive

### References

[__Programming in Scala, 
A comprehensive step-by-step guide__](https://www.artima.com/shop/programming_in_scala_3ed) Third Edition.
by Martin Odersky, Lex Spoon, and Bill Venners. 

- Chapter 16. Working with Lists
- Chapter 26. Extractors (optional)

[__Functional programming in Scala__](https://www.manning.com/books/functional-programming-in-scala), by Paul Chiusano and Runar Bjarnason.

- Chapter 3. Functional data structures

[__Functional programming simplified__](https://alvinalexander.com/downloads/fpsimplified-free-preview.pdf), by Alvin Alexander.

- Chapters 29-36. Recursion.

## Recursive types

### The `List` type

Lists are data structures which represent sequences of values of the same type, of finite length. They can be defined recursively in an informal way as follows: 
- A list is the empty sequence
- A list is a non-empty sequence made of a value and another list, which represent the head and tail of the list, respectively

Thus, the type `IntList`, which represents lists of integers, must satisfy the following algebraic equation:

`IntList = 1 + Int * IntList`

i.e., a list of integers is the empty sequence (represented by the singleton type `1`), or an integer (the head) and a list (its tail).



The implementation in Scala is similar to the following one (we also give the generic version `List[A]`, rather than the implementation of `IntList`):

In [2]:
object StdDefinition:
    enum List[X]: 
        case Empty()
        case NonEmpty(head: X, tail: List[X])

    import List._

    // [1,2,3,4]
    val l: List[Int] = 
        NonEmpty(1, 
            NonEmpty(2, 
                NonEmpty(3, 
                    NonEmpty(4, 
                        Empty() : List[Int]))))

defined object StdDefinition

However the actual implementation of [immutable lists](https://github.com/scala/scala/blob/v2.13.1/src/library/scala/collection/immutable/List.scala#L79) in the standard library of Scala defines the empty list as an object, rather than a class. This forces us to declare the list covariantly in its generic parameter `A`, which is somewhat inconvenient at times.  The standard definition looks like as follows:

In [3]:
object ActualStdDefinition:
    enum List[+X]:
        case Nil
        case ::(head: X, tail: List[X])

defined object ActualStdDefinition

### Some syntactic sugar

Note that we can write standard lists with a more compact syntax: 

In [9]:
// Less beautifully 
val l: List[Char] = ::('a', ::('b', ::('c', Nil)))
// More idiomatically
val l1: List[Char] = 'a' :: ('b' :: ('c' :: Nil))
val l2: List[Char] = 'a' :: 'b' :: 'c' :: Nil

val l3: List[Char] = List('a', 'b', 'c')
val l4: List[Char] = List()

l: List[Char] = List('a', 'b', 'c')
l1: List[Char] = List('a', 'b', 'c')
l2: List[Char] = List('a', 'b', 'c')
l3: List[Char] = List('a', 'b', 'c')
l4: List[Char] = List()

And we can also pattern match on lists, similarly:

In [12]:
// Less beautifully
List(1,2,3,4) match 
    case Nil => "empty"
    case ::(h, t) => 
        "non-empty with head " + h + " and tail " + t

// more idiomatically

List(1,2,3,4) match 
    case Nil => "empty"
    case h :: t => 
        "non-empty with head " + h + " and tail " + t

// or

List(1,2,3,4) match 
    case Nil => "empty"
    case List(x1, x2) => "two elements"
    case List(x1, x2, x3) => "three elements"
    case _ => "one element or more than three"


res12_0: String = "non-empty with head 1 and tail List(2, 3, 4)"
res12_1: String = "non-empty with head 1 and tail List(2, 3, 4)"
res12_2: String = "one element or more than three"

In [17]:
List(1,2,3).head
List(1,2,3).tail
List().tail

java.lang.UnsupportedOperationException: tail of empty list

##  Recursive functions

Since lists are defined recursively, functions over lists will be commonly recursive as well. For instance, let's implement a recursive function that computes the length of a list. But before, let's implement the function imperatively for the sake of comparison:

In [18]:
// Using mutable variables
def indexInt(l: List[Int], n: Int): Int = 
    ??? 

def indexChar(l: List[Char], n: Int): Char = 
    ??? 

def index[A](l: List[A], n: Int): Option[A] = 
    var i: Int = 0
    var aux: List[A] = l
    while (aux != Nil && i < n)
        aux = aux.tail
        i += 1
    if (aux == Nil) None
    else Some(aux.head: A)


defined function indexInt
defined function indexChar
defined function index

In [21]:
// invoke
index(List(0,1,2,3),0)
index(List(0,1,2,3),1)
index(List(0,1,2,3),2)
index(List(0,1,2,3),3)
index(List(0,1,2,3),4)




res21_0: Option[Int] = Some(value = 0)
res21_1: Option[Int] = Some(value = 1)
res21_2: Option[Int] = Some(value = 2)
res21_3: Option[Int] = Some(value = 3)
res21_4: Option[Int] = None

The recursive function is implemented as follows: 

In [24]:
// Using recursive functions

def index[A](list: List[A], n: Int): Option[A] = 
    list match
        case Nil => None : Option[A]
        case h :: t => 
            val tailSol: Option[A] = index(t, n-1)
            if (n == 0) Some(h)
            else tailSol : Option[A]

defined function index

In [29]:
// Using recursive functions

def index[A](list: List[A], n: Int): Option[A] = 
    list match
        case Nil => None : Option[A]
        case h :: t => 
            if (n == 0) Some(h)
            else index(t, n-1) : Option[A]

defined function index

In [30]:
// Using recursive functions

def index[A](list: List[A], n: Int): Option[A] = 
    list match
        case Nil => None : Option[A]
        case h :: t if n == 0 => Some(h)
        case _ :: t => index(t, n-1)

defined function index

In [31]:
index(List(0,1,2,3), 4)

res31: Option[Int] = None

Some comments: 
- The recursive function is implemented in a _type-driven development_ style: we proceed, step-by-step, analysing the types of input data that are available, and the types of output that we have to generate. This leads to a divide-and-conquer problem solving strategy and hugely facilitates the implementation.
- The recursive function is less efficient, since the stack will blow up with very long lists.

In [55]:
def lengthI[A](l: List[A]): Int = 
    var out: Int = 0 
    var aux: List[A] = l
    while (aux != Nil)
        aux = aux.tail
        out += 1
    out

defined function lengthI

In [56]:
lengthI(List.fill(16666666)(0))

res56: Int = 16666666

In [34]:
length(List()) 
length(List(1,2,3))

res34_0: Int = 0
res34_1: Int = 3

In [37]:
def length[A](l: List[A]): Int = 
    ???

defined function length

In [36]:
def length[A](l: List[A]): Int = 
    l match 
        case Nil => ??? : Int
        case h :: t => ??? : Int

defined function length

In [38]:
def length[A](l: List[A]): Int = 
    l match 
        case Nil => 0 : Int
        case h :: t => ??? : Int

defined function length

In [39]:
def length[A](l: List[A]): Int = 
    l match 
        case Nil => 0 : Int
        case h :: t => 
            val tailSol: Int = length(t)
            ??? : Int

defined function length

In [41]:
def length[A](l: List[A]): Int = 
    l match 
        case Nil => 0 : Int
        case h :: t => 
            val tailSol: Int = length(t)
            tailSol + 1 : Int

defined function length

In [49]:
List.fill(10)(List(true, false))

res49: List[List[Boolean]] = List(
  List(true, false),
  List(true, false),
  List(true, false),
  List(true, false),
  List(true, false),
  List(true, false),
  List(true, false),
  List(true, false),
  List(true, false),
  List(true, false)
)

In [54]:
Int.MaxValue

res54: Int = 2147483647

In [53]:
length(List.fill(1500000)('a'))

java.lang.StackOverflowError: null

### Tail-recursive functions

The implementation using tail-recursion solves the issues with the stack. It commonly makes use of auxiliary functions:

In [8]:
// Using tail-recursive functions



defined function lengthTR

res9_0: Int = 0
res9_1: Int = 3

We can check the stack-safety problems of non-tail recursive functions by calculating the length of a very big list. We will use the following function, which creates a constant list of given length.

In [10]:
// First, imperatively



defined function constantList

In [11]:
// Next, tail-recursively



defined function constantList

We can also use the function [`fill`](https://www.scala-lang.org/api/2.13.3/scala/collection/immutable/List$.html#fill[A](n:Int)(elem:=%3EA):CC[A]) of the Scala standard library.

Now, let's calculate the length of a list long enough to blow up the stack, using each of the three implementations:

In [12]:
// Imperatively


res12: Int = 100000

In [13]:
// Tail-recursive


res13: Int = 100000

In [13]:
// Plain recursive


### Unit testing with `scalatest`

In [1]:
import $ivy.`org.scalatest::scalatest:3.2.16`
import org.scalatest.{Filter => _, _}, flatspec._, matchers._

import $ivy.$                                

import org.scalatest.{Filter => _, _}, flatspec._, matchers._


From now on, we will also make extensive use of unit testing for the different functions that we implement. And we will use the [`scalatest`](http://www.scalatest.org/) library for that purpose. In particular, for each function we will implement a test catalogue that test it against different test cases. The test catalogue receives the actual function to be tested as a parameter. For instance, this is a possible test class for the `length` function:

In [ ]:
class TestLength(lengthF: List[Int] => Int) extends AnyFlatSpec with should.Matchers:
    "length" should "work" in:
        ??? 

The method `shouldBe` is a _matcher_. The scalatest library offers an extensive catalogue of [them](http://www.scalatest.org/user_guide/using_matchers). Similarly, scalatest also support many different [testing styles](http://www.scalatest.org/user_guide/selecting_a_style). The chosen one here was `FlatSpec`. In order to execute the test catalogue we can simply use the scalatest method `run`:

In [16]:
run(TestLength(lengthR))

cell15$Helper$TestLength:
length
- should work


### Example: adding numbers

In [2]:
object std: 
    enum Either[X, Y]: 
        case Left(x: X)
        case Right(y: Y)

    enum List[X]: 
        case Nil // 1
        case ::(head: X, tail: List[X]) // (X, List[X])

defined object std

In [5]:
val l: List[Int] = List(1,2,3,4)
val l1: List[Int] = 1 :: (2 :: (3 :: (4 :: Nil)))

val l2: List[Int] = ::(1, ::(2, ::(3, ::(4, Nil))))
val e: Either[String, Boolean] = Right(true)

l: List[Int] = List(1, 2, 3, 4)
l1: List[Int] = List(1, 2, 3, 4)
l2: List[Int] = List(1, 2, 3, 4)
e: Either[String, Boolean] = Right(value = true)

In [7]:
def isEmpty[X](l: List[X]): Boolean = 
    l match 
        case Nil => true : Boolean
        case head :: tail => false : Boolean

defined function isEmpty

Let's implement a function that sums all the numbers of a list.

In [8]:
def sum(l: List[Int]): Int = 
    var out: Int = ??? 
    for (e <- l)
        out = ???(out, e)
    out

defined function sum

In [21]:
def sumI(l: List[Int]): Int = 
    var out: Int = 0
    for (e <- l)
        out = out + e
    /*return*/ out

defined function sumI

In [22]:
class TestSum(sum: List[Int] => Int) 
extends AnyFlatSpec with should.Matchers:
    "sum" should "return 0 for the empty list" in:
        sum(Nil) shouldBe 0 

    "sum" should "return the right value for non-empty lists" in:
        sum(1 :: Nil) shouldBe 1
        sum(List(6, 2, 3)) shouldBe 11 

defined class TestSum

In [23]:
run(TestSum(sumI))

cell22$Helper$TestSum:
sum
- should return 0 for the empty list
sum
- should return the right value for non-empty lists


In [24]:
def sum(l: List[Int]): Int = 
   ??? : Int

defined function sum

In [24]:
def sum(l: List[Int]): Int = 
    l match 
        case Nil => ??? : Int
        case head :: tail => ??? : Int

defined function sum

In [24]:
def sum(l: List[Int]): Int = 
    l match 
        case Nil => 
            0 : Int
        case (head: Int) :: (tail: List[Int]) => 
            ??? : Int

defined function sum

In [68]:
@scala.annotation.tailrec
def sumR(l: List[Int]): Int = 
    l match 
        case Nil => 
            0 : Int
        case (head: Int) :: (tail: List[Int]) => 
            val tailSum: Int = sumR(tail)
            head + tailSum : Int

-- Error: cell69.sc:7:35 -------------------------------------------------------
7 |            val tailSum: Int = sumR(tail)
  |                               ^^^^^^^^^^
  |                 Cannot rewrite recursive call: it is not in tail position
Compilation Failed

In [50]:
def sumR(l: List[Int]): Int = 
    l match 
        case Nil => 
            0 : Int
        case head :: tail => 
            head + sumR(tail)

defined function sumR

In [51]:
run(TestSum(sumR))

cell22$Helper$TestSum:
sum
- should return 0 for the empty list
sum
- should return the right value for non-empty lists


In [28]:
// Recursively



In [19]:
run(TestSum(sum))

cell17$Helper$TestSum:
sum
- should work


In [20]:
// With tail-recursion



defined function sumTR

In [68]:
def sumTR(l: List[Int]): Int = 

    @scala.annotation.tailrec
    def sumAux(out: Int, aux: List[Int]): Int = 
        aux match 
            case Nil => out
            case head :: tail => 
                sumAux(out + head, tail) 

    sumAux(0, l)
        
    /*var out: Int = 0
    var aux: List[Int] = l
    while (aux != Nil)
        out = out + aux.head
        aux = aux.tail
    out*/

    
    

defined function sumTR

In [67]:
run(TestSum(sumTR))

cell22$Helper$TestSum:
sum
- should return 0 for the empty list
sum
- should return the right value for non-empty lists


In [72]:
sumTR(List.fill(10000000)(1))

res72: Int = 10000000

In [56]:
List.fill(10)(1)

res56: List[Int] = List(1, 1, 1, 1, 1, 1, 1, 1, 1, 1)

In [63]:
//sumI(List.fill(10000)(1))
sumR(List.fill(100000)(1))

java.lang.StackOverflowError: null

### Example: multiplying list elements

Let's multiply the elements of a list. If the list is empty we return the identity element for integers.

In [22]:
class TestProduct(product: List[Int] => Int) extends AnyFlatSpec with should.Matchers:
    "product" should "work" in:
        ???

defined class TestProduct

 This is the common recursive implementation:

defined function product

In [24]:
run(TestProduct(product))

cell22$Helper$TestProduct:
product
- should work


But we can optimize the function a little bit. Note that if the number 0 belongs to the list, then the result is 0, no matter how many elements the list has. So, once we find the element 0 it's a waste of resources to make the recursive call. Let's take this into account.

In [25]:
// optimization for 0



defined function product2

In [26]:
run(TestProduct(product2))

cell22$Helper$TestProduct:
product
- should work


A similar optimization can be made for the tail-recursive implementation.

### Example: membership

Let's implement a function that given a list and an element, returns whether the element belongs to that list.

In [30]:
class TestMember(member: (List[Int], Int) => Boolean) extends AnyFlatSpec with should.Matchers:
    "member" should "work" in:
        member(Nil, 6) shouldBe false 
        member(List(1, 8 ,5), 6) shouldBe false 
        member(List(1, 8 ,5), 1) shouldBe true 
        member(List(1, 8 ,5), 5) shouldBe true 
        member(List(1, 8 ,5), 8) shouldBe true 

defined class TestMember

In [33]:
def member(l: List[Int], e: Int): Boolean = 
    l match 
        case Nil => ??? : Boolean
        case head :: tail => 
            val tailSol: Boolean = ??? 
            ??? : Boolean 

defined function member

In [36]:
def memberR(l: List[Int], e: Int): Boolean = 
    l match 
        case Nil => false : Boolean
        case head :: tail => 
            val tailSol: Boolean = memberR(tail, e)
            /*if (head == e) true 
            else tailSol : Boolean */
            (head == e) || tailSol

defined function memberR

In [36]:
def memberR(l: List[Int], e: Int): Boolean = 
    l match 
        case Nil => false : Boolean
        case head :: tail => 
            (head == e) || memberR(tail, e)

defined function memberR

In [36]:
def memberR(l: List[Int], e: Int): Boolean = 
    l match 
        case Nil => false : Boolean
        case head :: tail if head == e => true
        case _ :: tail => memberR(tail, e)

defined function memberR

In [47]:
def memberR(l: List[Int], e: Int): Boolean = 
    l match 
        case Nil => false : Boolean
        case `e` :: tail => true
        case _ :: tail => memberR(tail, e)

defined function memberR

In [48]:
run(TestMember(memberR))

cell30$Helper$TestMember:
member
- should work


In [38]:
def memberI(l: List[Int], e: Int): Boolean = 
    var out: Boolean = ???
    for (e <- l)
        out = ???(out, e)
    out

defined function memberI

In [45]:
def memberI(l: List[Int], e: Int): Boolean = 
    var out: Boolean = false
    var aux: List[Int] = l
    while (aux != Nil && !out)
        out = out || aux.head == e
        aux = aux.tail
    out

defined function memberI

In [39]:
def memberI(l: List[Int], e: Int): Boolean = 
    var out: Boolean = false
    for (e1 <- l)
        out = out || e1 == e
    out

defined function memberI

In [46]:
run(TestMember(memberI))

cell30$Helper$TestMember:
member
- should work


We can also pattern match against a specific value as follows:

defined function member

### Example: last element

Let's implement a function that returns the last element of a given list. Note that an empty list does not have elements, and, hence, does not have a last element.

In [31]:
class TestLast(last: List[Int] => Option[Int]) extends AnyFlatSpec with should.Matchers:
    "last" should "work" in:
        ???

defined class TestLast

In [32]:
@annotation.tailrec


defined function last

In [33]:
run(TestLast(last))

cell31$Helper$TestLast:
last
- should work


### Example: insert last

Now, a function that allows us to insert an element at the end of the list. 

In [12]:
class TestInsertLast(insertLast: (List[Int], Int) => List[Int]) 
extends AnyFlatSpec with should.Matchers:
    "insertLast" should "work" in:
        insertLast(List(1,2,3), 4) shouldBe List(1,2,3,4)
        insertLast(1::List(2,3), 4) shouldBe 1::List(2,3,4)
        insertLast(List(), 1) shouldBe List(1)

defined class TestInsertLast

In [13]:
def insertLast[A](l: List[A], e: A): List[A] = 
    l match 
        case Nil => ??? : List[A] 
        case head :: tail => 
            val tailSol: List[A] = insertLast(tail, e)
            ??? : List[A]

defined function insertLast

In [14]:
def insertLast[A](l: List[A], e: A): List[A] = 
    l match 
        case Nil => List(e) : List[A] 
        case head :: tail => 
            val tailSol: List[A] = insertLast(tail, e)
            head :: tailSol : List[A]

defined function insertLast

In [15]:
run(TestInsertLast(insertLast))

cell12$Helper$TestInsertLast:
insertLast
- should work


### Example: reverse lists

Implement a function which receives a list and returns its reverse.

In [18]:
List(1,2,3) :+ 4

res18: List[Int] = List(1, 2, 3, 4)

In [17]:
class TestReverse(reverse: List[Int] => List[Int]) extends AnyFlatSpec with should.Matchers:
    "reverse" should "work" in:
        reverse(List(1,2,3,4)) shouldBe List(4,3,2,1)
        reverse(1::List(2,3,4)) shouldBe insertLast(List(4,3,2), 1)
        reverse(List()) shouldBe List()
        reverse(List(7,3,2)) shouldBe List(2,3,7)

defined class TestReverse

In [20]:
// Really inefficient 

def reverse(l: List[Int]): List[Int] = 
    l match 
        case Nil => Nil 
        case head :: tail => 
            val tailSol: List[Int] = reverse(tail)
            insertLast(tailSol, head) : List[Int]

defined function reverse

In [23]:
// Really inefficient 

def reverse(l: List[Char]): List[Char] = 
    l match 
        case Nil => Nil 
        case head :: tail => insertLast(reverse(tail), head)

defined function reverse

In [26]:
// Really inefficient 

def reverse[A](l: List[A]): List[A] = 
    l match 
        case Nil => Nil 
        case head :: tail => insertLast(reverse(tail), head)

defined function reverse

In [34]:
reverse(List.fill(1000000)(0))

java.lang.StackOverflowError: null

In [ ]:
def reverseI[A](l: List[A]): List[A] = 
    var out: List[A] = ??? 
    var aux: List[A] = l
    while (aux != Nil)
        out = ?(out, aux.head)
        aux = aux.tail
    out

In [35]:
def reverseI[A](l: List[A]): List[A] = 
    var out: List[A] = Nil
    var aux: List[A] = l
    while (aux != Nil)
        out = aux.head :: out
        aux = aux.tail
    out

defined function reverseI

In [40]:
def reverseTR[A](l: List[A]): List[A] = 
    
    def rev_aux(aux: List[A], out: List[A]): List[A] = 
        aux match 
            case Nil => out
            case head :: tail => 
                rev_aux(tail, head :: out) 

    rev_aux(l, Nil)
/*
    var out: List[A] = Nil
    var aux: List[A] = l
    while (aux != Nil)
        out = aux.head :: out
        aux = aux.tail
    out*/

defined function reverseTR

In [35]:
def reverseTR[A](l: List[A]): List[A] = 
    
    def rev_aux(aux: List[A], out: List[A]): List[A] = 
        aux match 
            case Nil => out 
            case head :: tail => 
                rev_aux(tail, ?(out, head))

    rev_aux(l, ???)
/*
    var out: List[A] = Nil
    var aux: List[A] = l
    while (aux != Nil)
        out = aux.head :: out
        aux = aux.tail
    out*/

defined function reverseI

In [42]:
reverseTR(List.fill(1000000)(0))

res42: List[Int] = List(
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
...

In [19]:
// Really inefficient 

def reverse(l: List[Int]): List[Int] = 
    l match 
        case Nil => ??? 
        case head :: tail => 
            val tailSol: List[Int] = ??? 
            ??? : List[Int]

defined function reverse

In [28]:
run(TestReverse(reverse[Int]))

cell17$Helper$TestReverse:
reverse
- should work


In [36]:
run(TestReverse(reverseI[Int]))

cell17$Helper$TestReverse:
reverse
- should work


In [41]:
run(TestReverse(reverseTR[Int]))

cell17$Helper$TestReverse:
reverse
- should work


In [53]:
// Tail-recursive, efficiently



defined function reverseTR

In [54]:
run(TestReverse(reverseTR))

cell50$Helper$TestReverse:
reverse
- should work


### Example: concatenate lists

In [43]:
class TestConcatenate(concatenate: (List[Int], List[Int]) => List[Int]) 
extends AnyFlatSpec with should.Matchers:
    "concatenate" should "work" in:
        concatenate(List(1,2,3), List(4,5,6)) shouldBe List(1,2,3,4,5,6)
        concatenate(List(), List(1,2,3)) shouldBe List(1,2,3)
        concatenate(List(), List()) shouldBe List()
        concatenate(List(1,2,3), List()) shouldBe List(1,2,3)

defined class TestConcatenate

In [48]:
def concatenate[A](l1: List[A], l2: List[A]): List[A] = 
  l1 match 
    case Nil => ???
    case h :: t => 
        val tailSol: List[A] = ??? 
        ???

defined function concatenate

In [52]:
// NO @scala.annotation.tailrec 
def concatenate[A](l1: List[A], l2: List[A]): List[A] = 
  l1 match 
    case Nil => l2
    case h :: t => 
        val tailSol: List[A] = concatenate(t, l2) 
        h :: tailSol

defined function concatenate

In [53]:
// NO @scala.annotation.tailrec 
def concatenate[A](l1: List[A], l2: List[A]): List[A] = 
  l1 match 
    case Nil => l2
    case h :: t => h :: concatenate(t, l2) 

defined function concatenate

In [48]:
def concatenate[A](l1: List[A], l2: List[A]): List[A] = 
    
    @scala.annotation.tailrec
    def conc_aux(aux: List[A], out: List[A]): List[A] = 
        aux match 
            case Nil => out
            case e :: tail => 
                conc_aux(tail, e :: out)

    conc_aux(conc_aux(l1, Nil), l2)

defined function concatenate

In [51]:
run(TestConcatenate(concatenate))

cell43$Helper$TestConcatenate:
concatenate
- should work


Tail-recursive concatenation:

defined function concatenateTR

In [60]:
run(TestConcatenate(concatenateTR))

cell56$Helper$TestConcatenate:
concatenate
- should work
